In [ ]:
import pandas as pd
import numpy as np

ruta = "C:\\Users\\CO-166\\Desktop\\AdquisInfo\\baseDatosAC.csv"
tabla = pd.read_csv(ruta)
display(tabla)


C:\Users\CO-166\AppData\Local\Temp\ipykernel_1468\2939087558.py:5: DtypeWarning: Columns (10,13,17,24) have mixed types. Specify dtype option on import or set low_memory=False.
  tabla = pd.read_csv(ruta)


,Nombres y Apellidos,Fecha de Nacimiento,Genero de nacimiento,Estado Civil,Nivel Educativo,Empresa,Sede,Cargo,Ciudad de Residencia,Tipo de Documento,...,Pruebas psicotécnicas,Prueba 1,Prueba 2,Prueba 3,Experiencia - Conocimientos,Habilidades,Contratado,Pruebas psicotécnicas.1,Desarrollo Familiar,Tipo de proceso
0,JAIME ALBERTO RAMIREZ CUERVO,13/07/1995,MASCULINO,NaN,NaN,SECURITY SHOPS LIMITADA,SECURITY SHOPS - SEDE PRINCIPAL BOGOTA,TÉCNICO DE SOSTENIMIENTO,"BOGOTA, D.C.",CEDULA DE CIUDADANIA,...,NaN,NaN,NaN,NaN,1 año de EXPERIENCIA ESPECÍFICA EN Empresa del...,1 año de EXPERIENCIA ESPECÍFICA EN Empresa del...,False,"[""Valores"",""NIC""]",NaN,Proceso de selección
1,JAIDER NAYID LOPEZ BLANCO,02/06/2003,MASCULINO,SOLTERO(A),MEDIA ACADEMICA (BACHILLER),OXIGENO CERO GRADOS S.A.S,OXIGENO CERO GRADOS - SEDE PRINCIPAL BOGOTA,BODEGUERO,"BOGOTA, D.C.",CEDULA DE CIUDADANIA,...,NaN,NaN,NaN,NaN,NA,NA,False,"[""Ninguna""]",JAIDER NAYID LOPEZ BLANCO actualmente tioenw...,Referido
2,YORYIS BARROS GUTIÉRREZ,04/04/1997,FEMENINO,SOLTERO(A),MEDIA ACADEMICA (BACHILLER),DAMIS SAS,BARRANQUILLA,INSTALADOR,BARRANQUILLA,CEDULA DE CIUDADANIA,...,NaN,NaN,NaN,NaN,6 meses de experiencia en funciones como Insta...,6 meses de experiencia en funciones como Insta...,False,"[""Valores""]",NaN,Proceso de selección
3,OSWALDO ANTINIO JIMENEZ RIBON,14/12/1996,MASCULINO,SOLTERO(A),TECNICA PROFESIONAL,DAMIS SAS,BARRANQUILLA,INSTALADOR,BARRANQUILLA,CEDULA DE CIUDADANIA,...,NaN,NaN,NaN,NaN,6 meses de experiencia en funciones como Insta...,6 meses de experiencia en funciones como Insta...,False,"[""Valores""]",NaN,Proceso de selección
4,YESSIKA JULIANA PACHECO BETANCUR,27/10/1995,FEMENINO,SOLTERO(A),TECNICA PROFESIONAL,AUDIFARMA S.A.,ALCALA,AUXILIAR CAF,ALCALA,CEDULA DE CIUDADANIA,...,NaN,NaN,NaN,NaN,6 meses de experiencia en dispensación de medi...,6 meses de experiencia en dispensación de medi...,False,"[""Valores"",""NIC""]",YESSIKA JULIANA PACHECO BETANCUR actualmente t...,Referido
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79442,RONALD DAVIAN DELGADO VEGA,21/11/1988,MASCULINO,UNION LIBRE,NINGUNO,TREFILADOS DE COLOMBIA S.A.S.,BODEGA MOSQUERA,OPERARIO,MOSQUERA,CEDULA DE CIUDADANIA,...,NaN,NaN,NaN,NaN,1 AÑO,1 AÑO,True,NaN,"ACTUALMENTE CONVIVE EN UNIÓN LIBRE, LA RELACIÓ...",NaN
79443,ALFONSO RAFAEL CANDAMA BROCHERO,14/09/1983,MASCULINO,SOLTERO(A),NINGUNO,TREFILADOS DE COLOMBIA S.A.S.,BODEGA MOSQUERA,OPERARIO,MOSQUERA,CEDULA DE CIUDADANIA,...,NaN,NaN,NaN,NaN,1 AÑO,1 AÑO,True,NaN,"EL CANDIDATO ES SOLTERO, DEFINE SU FAMILIA COM...",NaN
79444,Jhonathan Danilo Buyucue Mosquera,01/06/1993,MASCULINO,UNION LIBRE,NINGUNO,TREFILADOS DE COLOMBIA S.A.S.,BODEGA MOSQUERA,OPERARIO,MOSQUERA,CEDULA DE CIUDADANIA,...,NaN,NaN,NaN,NaN,1 AÑO,1 AÑO,True,NaN,NaN,NaN
79445,HAROLD ANDRES ALZATE GONZALEZ,30/01/1984,MASCULINO,UNION LIBRE,MEDIA ACADEMICA (BACHILLER),TREFILADOS DE COLOMBIA S.A.S.,BODEGA MOSQUERA,OPERARIO,MOSQUERA,CEDULA DE CIUDADANIA,...,NaN,NaN,NaN,NaN,1 AÑO,1 AÑO,True,NaN,SU NUCLEO FAMILIAR ES ESTABLE BASADO EN EL RES...,NaN


In [ ]:
tabla["Documento"].value_counts(dropna=False)

1030611246    48
1013677246    33
1024585888    29
1233688144    27
1023029933    27
              ..
1000789842     1
1233902404     1
1118291059     1
14698068       1
1075657075     1
Name: Documento, Length: 60909, dtype: int64

In [ ]:
tabla["Prueba 2"].describe

<bound method NDFrame.describe of 0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
79442    NaN
79443    NaN
79444    NaN
79445    NaN
79446    NaN
Name: Prueba 2, Length: 79447, dtype: object>

In [ ]:
lista = []
lista2 = []
lista3=[]
listaGen=[]
for x in tabla.index:
    if str(tabla["Prueba 1"][x]).__contains__("."):
        lista.append(1)
    else:
        lista.append(0)

    if str(tabla["Prueba 2"][x]).__contains__("."):
        lista2.append(1)
    else:
        lista2.append(0)

    if str(tabla["Prueba 3"][x]).__contains__("."):
        lista3.append(1)
        
    else:
        lista3.append(0)

    if str(tabla["Genero de nacimiento"][x]).__contains__("MASC"):
        listaGen.append(1)
        
    else:
        listaGen.append(0)    


dataFrame = pd.DataFrame(lista, columns=["PruebaCont_01"])
dataFrame1 = pd.DataFrame(lista2, columns=["PruebaCont_02"])
dataFrame2 = pd.DataFrame(lista3, columns=["PruebaCont_03"])
dataGen = pd.DataFrame(listaGen, columns=["Genero"])

tabla2 = pd.concat([tabla, dataFrame,dataFrame1,dataFrame2, dataGen], axis=1)
tabla2


,Nombres y Apellidos,Fecha de Nacimiento,Genero de nacimiento,Estado Civil,Nivel Educativo,Empresa,Sede,Cargo,Ciudad de Residencia,Tipo de Documento,...,Experiencia - Conocimientos,Habilidades,Contratado,Pruebas psicotécnicas.1,Desarrollo Familiar,Tipo de proceso,PruebaCont_01,PruebaCont_02,PruebaCont_03,Genero
0,JAIME ALBERTO RAMIREZ CUERVO,13/07/1995,MASCULINO,NaN,NaN,SECURITY SHOPS LIMITADA,SECURITY SHOPS - SEDE PRINCIPAL BOGOTA,TÉCNICO DE SOSTENIMIENTO,"BOGOTA, D.C.",CEDULA DE CIUDADANIA,...,1 año de EXPERIENCIA ESPECÍFICA EN Empresa del...,1 año de EXPERIENCIA ESPECÍFICA EN Empresa del...,False,"[""Valores"",""NIC""]",NaN,Proceso de selección,0,0,0,1
1,JAIDER NAYID LOPEZ BLANCO,02/06/2003,MASCULINO,SOLTERO(A),MEDIA ACADEMICA (BACHILLER),OXIGENO CERO GRADOS S.A.S,OXIGENO CERO GRADOS - SEDE PRINCIPAL BOGOTA,BODEGUERO,"BOGOTA, D.C.",CEDULA DE CIUDADANIA,...,NA,NA,False,"[""Ninguna""]",JAIDER NAYID LOPEZ BLANCO actualmente tioenw...,Referido,0,0,0,1
2,YORYIS BARROS GUTIÉRREZ,04/04/1997,FEMENINO,SOLTERO(A),MEDIA ACADEMICA (BACHILLER),DAMIS SAS,BARRANQUILLA,INSTALADOR,BARRANQUILLA,CEDULA DE CIUDADANIA,...,6 meses de experiencia en funciones como Insta...,6 meses de experiencia en funciones como Insta...,False,"[""Valores""]",NaN,Proceso de selección,0,0,0,0
3,OSWALDO ANTINIO JIMENEZ RIBON,14/12/1996,MASCULINO,SOLTERO(A),TECNICA PROFESIONAL,DAMIS SAS,BARRANQUILLA,INSTALADOR,BARRANQUILLA,CEDULA DE CIUDADANIA,...,6 meses de experiencia en funciones como Insta...,6 meses de experiencia en funciones como Insta...,False,"[""Valores""]",NaN,Proceso de selección,0,0,0,1
4,YESSIKA JULIANA PACHECO BETANCUR,27/10/1995,FEMENINO,SOLTERO(A),TECNICA PROFESIONAL,AUDIFARMA S.A.,ALCALA,AUXILIAR CAF,ALCALA,CEDULA DE CIUDADANIA,...,6 meses de experiencia en dispensación de medi...,6 meses de experiencia en dispensación de medi...,False,"[""Valores"",""NIC""]",YESSIKA JULIANA PACHECO BETANCUR actualmente t...,Referido,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79442,RONALD DAVIAN DELGADO VEGA,21/11/1988,MASCULINO,UNION LIBRE,NINGUNO,TREFILADOS DE COLOMBIA S.A.S.,BODEGA MOSQUERA,OPERARIO,MOSQUERA,CEDULA DE CIUDADANIA,...,1 AÑO,1 AÑO,True,NaN,"ACTUALMENTE CONVIVE EN UNIÓN LIBRE, LA RELACIÓ...",NaN,0,0,0,1
79443,ALFONSO RAFAEL CANDAMA BROCHERO,14/09/1983,MASCULINO,SOLTERO(A),NINGUNO,TREFILADOS DE COLOMBIA S.A.S.,BODEGA MOSQUERA,OPERARIO,MOSQUERA,CEDULA DE CIUDADANIA,...,1 AÑO,1 AÑO,True,NaN,"EL CANDIDATO ES SOLTERO, DEFINE SU FAMILIA COM...",NaN,0,0,0,1
79444,Jhonathan Danilo Buyucue Mosquera,01/06/1993,MASCULINO,UNION LIBRE,NINGUNO,TREFILADOS DE COLOMBIA S.A.S.,BODEGA MOSQUERA,OPERARIO,MOSQUERA,CEDULA DE CIUDADANIA,...,1 AÑO,1 AÑO,True,NaN,NaN,NaN,0,0,0,1
79445,HAROLD ANDRES ALZATE GONZALEZ,30/01/1984,MASCULINO,UNION LIBRE,MEDIA ACADEMICA (BACHILLER),TREFILADOS DE COLOMBIA S.A.S.,BODEGA MOSQUERA,OPERARIO,MOSQUERA,CEDULA DE CIUDADANIA,...,1 AÑO,1 AÑO,True,NaN,SU NUCLEO FAMILIAR ES ESTABLE BASADO EN EL RES...,NaN,0,0,0,1


In [ ]:
for x in tabla2.index:
    if tabla2["Empresa"][x].__contains__("EMPRESA DE PRUEBA"):
        tabla2.drop(index=x, inplace=True)
tabla2


,Nombres y Apellidos,Fecha de Nacimiento,Genero de nacimiento,Estado Civil,Nivel Educativo,Empresa,Sede,Cargo,Ciudad de Residencia,Tipo de Documento,...,Experiencia - Conocimientos,Habilidades,Contratado,Pruebas psicotécnicas.1,Desarrollo Familiar,Tipo de proceso,PruebaCont_01,PruebaCont_02,PruebaCont_03,Genero
0,JAIME ALBERTO RAMIREZ CUERVO,13/07/1995,MASCULINO,NaN,NaN,SECURITY SHOPS LIMITADA,SECURITY SHOPS - SEDE PRINCIPAL BOGOTA,TÉCNICO DE SOSTENIMIENTO,"BOGOTA, D.C.",CEDULA DE CIUDADANIA,...,1 año de EXPERIENCIA ESPECÍFICA EN Empresa del...,1 año de EXPERIENCIA ESPECÍFICA EN Empresa del...,False,"[""Valores"",""NIC""]",NaN,Proceso de selección,0,0,0,1
1,JAIDER NAYID LOPEZ BLANCO,02/06/2003,MASCULINO,SOLTERO(A),MEDIA ACADEMICA (BACHILLER),OXIGENO CERO GRADOS S.A.S,OXIGENO CERO GRADOS - SEDE PRINCIPAL BOGOTA,BODEGUERO,"BOGOTA, D.C.",CEDULA DE CIUDADANIA,...,NA,NA,False,"[""Ninguna""]",JAIDER NAYID LOPEZ BLANCO actualmente tioenw...,Referido,0,0,0,1
2,YORYIS BARROS GUTIÉRREZ,04/04/1997,FEMENINO,SOLTERO(A),MEDIA ACADEMICA (BACHILLER),DAMIS SAS,BARRANQUILLA,INSTALADOR,BARRANQUILLA,CEDULA DE CIUDADANIA,...,6 meses de experiencia en funciones como Insta...,6 meses de experiencia en funciones como Insta...,False,"[""Valores""]",NaN,Proceso de selección,0,0,0,0
3,OSWALDO ANTINIO JIMENEZ RIBON,14/12/1996,MASCULINO,SOLTERO(A),TECNICA PROFESIONAL,DAMIS SAS,BARRANQUILLA,INSTALADOR,BARRANQUILLA,CEDULA DE CIUDADANIA,...,6 meses de experiencia en funciones como Insta...,6 meses de experiencia en funciones como Insta...,False,"[""Valores""]",NaN,Proceso de selección,0,0,0,1
4,YESSIKA JULIANA PACHECO BETANCUR,27/10/1995,FEMENINO,SOLTERO(A),TECNICA PROFESIONAL,AUDIFARMA S.A.,ALCALA,AUXILIAR CAF,ALCALA,CEDULA DE CIUDADANIA,...,6 meses de experiencia en dispensación de medi...,6 meses de experiencia en dispensación de medi...,False,"[""Valores"",""NIC""]",YESSIKA JULIANA PACHECO BETANCUR actualmente t...,Referido,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79442,RONALD DAVIAN DELGADO VEGA,21/11/1988,MASCULINO,UNION LIBRE,NINGUNO,TREFILADOS DE COLOMBIA S.A.S.,BODEGA MOSQUERA,OPERARIO,MOSQUERA,CEDULA DE CIUDADANIA,...,1 AÑO,1 AÑO,True,NaN,"ACTUALMENTE CONVIVE EN UNIÓN LIBRE, LA RELACIÓ...",NaN,0,0,0,1
79443,ALFONSO RAFAEL CANDAMA BROCHERO,14/09/1983,MASCULINO,SOLTERO(A),NINGUNO,TREFILADOS DE COLOMBIA S.A.S.,BODEGA MOSQUERA,OPERARIO,MOSQUERA,CEDULA DE CIUDADANIA,...,1 AÑO,1 AÑO,True,NaN,"EL CANDIDATO ES SOLTERO, DEFINE SU FAMILIA COM...",NaN,0,0,0,1
79444,Jhonathan Danilo Buyucue Mosquera,01/06/1993,MASCULINO,UNION LIBRE,NINGUNO,TREFILADOS DE COLOMBIA S.A.S.,BODEGA MOSQUERA,OPERARIO,MOSQUERA,CEDULA DE CIUDADANIA,...,1 AÑO,1 AÑO,True,NaN,NaN,NaN,0,0,0,1
79445,HAROLD ANDRES ALZATE GONZALEZ,30/01/1984,MASCULINO,UNION LIBRE,MEDIA ACADEMICA (BACHILLER),TREFILADOS DE COLOMBIA S.A.S.,BODEGA MOSQUERA,OPERARIO,MOSQUERA,CEDULA DE CIUDADANIA,...,1 AÑO,1 AÑO,True,NaN,SU NUCLEO FAMILIAR ES ESTABLE BASADO EN EL RES...,NaN,0,0,0,1


In [ ]:
tabla2["Documento"].value_counts(dropna=False)

1030611246    48
1024585888    29
1007400940    27
1233688144    27
1023029933    27
              ..
1118291059     1
14698068       1
1144157021     1
1000620462     1
1075657075     1
Name: Documento, Length: 60864, dtype: int64

In [ ]:
tabla2["Nivel Educativo"] = np.where(
            (tabla2["Nivel Educativo"] == "NINGUNO"),
            0,
            np.where(
                (tabla2["Nivel Educativo"] == "BASICA PRIMARIA"),
                1,
                np.where(
                    (tabla2["Nivel Educativo"] == "MEDIA ACADEMICA (BACHILLER)"),
                    2,
                    np.where(
                        (tabla2["Nivel Educativo"] == "TECNICA PROFESIONAL"),
                        3,
                        np.where(
                            (tabla2["Nivel Educativo"] == "TECNOLOGICA"),
                            4, np.where(
                                (tabla2["Nivel Educativo"] == "PROFESIONAL"),
                                5,
                                np.where(
                                    (tabla2["Nivel Educativo"] == "POSTGRADO"),
                                    6,
                                    0
                                )
                            )
                        )
                    )
                )
            )
        )
tabla2

,Nombres y Apellidos,Fecha de Nacimiento,Genero de nacimiento,Estado Civil,Nivel Educativo,Empresa,Sede,Cargo,Ciudad de Residencia,Tipo de Documento,...,Experiencia - Conocimientos,Habilidades,Contratado,Pruebas psicotécnicas.1,Desarrollo Familiar,Tipo de proceso,PruebaCont_01,PruebaCont_02,PruebaCont_03,Genero
0,JAIME ALBERTO RAMIREZ CUERVO,13/07/1995,MASCULINO,NaN,0,SECURITY SHOPS LIMITADA,SECURITY SHOPS - SEDE PRINCIPAL BOGOTA,TÉCNICO DE SOSTENIMIENTO,"BOGOTA, D.C.",CEDULA DE CIUDADANIA,...,1 año de EXPERIENCIA ESPECÍFICA EN Empresa del...,1 año de EXPERIENCIA ESPECÍFICA EN Empresa del...,False,"[""Valores"",""NIC""]",NaN,Proceso de selección,0,0,0,1
1,JAIDER NAYID LOPEZ BLANCO,02/06/2003,MASCULINO,SOLTERO(A),2,OXIGENO CERO GRADOS S.A.S,OXIGENO CERO GRADOS - SEDE PRINCIPAL BOGOTA,BODEGUERO,"BOGOTA, D.C.",CEDULA DE CIUDADANIA,...,NA,NA,False,"[""Ninguna""]",JAIDER NAYID LOPEZ BLANCO actualmente tioenw...,Referido,0,0,0,1
2,YORYIS BARROS GUTIÉRREZ,04/04/1997,FEMENINO,SOLTERO(A),2,DAMIS SAS,BARRANQUILLA,INSTALADOR,BARRANQUILLA,CEDULA DE CIUDADANIA,...,6 meses de experiencia en funciones como Insta...,6 meses de experiencia en funciones como Insta...,False,"[""Valores""]",NaN,Proceso de selección,0,0,0,0
3,OSWALDO ANTINIO JIMENEZ RIBON,14/12/1996,MASCULINO,SOLTERO(A),3,DAMIS SAS,BARRANQUILLA,INSTALADOR,BARRANQUILLA,CEDULA DE CIUDADANIA,...,6 meses de experiencia en funciones como Insta...,6 meses de experiencia en funciones como Insta...,False,"[""Valores""]",NaN,Proceso de selección,0,0,0,1
4,YESSIKA JULIANA PACHECO BETANCUR,27/10/1995,FEMENINO,SOLTERO(A),3,AUDIFARMA S.A.,ALCALA,AUXILIAR CAF,ALCALA,CEDULA DE CIUDADANIA,...,6 meses de experiencia en dispensación de medi...,6 meses de experiencia en dispensación de medi...,False,"[""Valores"",""NIC""]",YESSIKA JULIANA PACHECO BETANCUR actualmente t...,Referido,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79442,RONALD DAVIAN DELGADO VEGA,21/11/1988,MASCULINO,UNION LIBRE,0,TREFILADOS DE COLOMBIA S.A.S.,BODEGA MOSQUERA,OPERARIO,MOSQUERA,CEDULA DE CIUDADANIA,...,1 AÑO,1 AÑO,True,NaN,"ACTUALMENTE CONVIVE EN UNIÓN LIBRE, LA RELACIÓ...",NaN,0,0,0,1
79443,ALFONSO RAFAEL CANDAMA BROCHERO,14/09/1983,MASCULINO,SOLTERO(A),0,TREFILADOS DE COLOMBIA S.A.S.,BODEGA MOSQUERA,OPERARIO,MOSQUERA,CEDULA DE CIUDADANIA,...,1 AÑO,1 AÑO,True,NaN,"EL CANDIDATO ES SOLTERO, DEFINE SU FAMILIA COM...",NaN,0,0,0,1
79444,Jhonathan Danilo Buyucue Mosquera,01/06/1993,MASCULINO,UNION LIBRE,0,TREFILADOS DE COLOMBIA S.A.S.,BODEGA MOSQUERA,OPERARIO,MOSQUERA,CEDULA DE CIUDADANIA,...,1 AÑO,1 AÑO,True,NaN,NaN,NaN,0,0,0,1
79445,HAROLD ANDRES ALZATE GONZALEZ,30/01/1984,MASCULINO,UNION LIBRE,2,TREFILADOS DE COLOMBIA S.A.S.,BODEGA MOSQUERA,OPERARIO,MOSQUERA,CEDULA DE CIUDADANIA,...,1 AÑO,1 AÑO,True,NaN,SU NUCLEO FAMILIAR ES ESTABLE BASADO EN EL RES...,NaN,0,0,0,1


In [ ]:
def normalize(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
        (" ", "_"),
        ("ñ","n"),
        ("(", ""),
        (")", ""),
        (".","_")
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.lower(), b.lower()).lower()
    return s


column_names = []
for x in tabla2.columns:
     y = normalize(x)
     column_names.append(y)

tabla2.columns = column_names
tabla2.head(5)

tabla2.columns

Index(['nombres_y_apellidos', 'fecha_de_nacimiento', 'genero_de_nacimiento',
       'estado_civil', 'nivel_educativo', 'empresa', 'sede', 'cargo',
       'ciudad_de_residencia', 'tipo_de_documento', 'documento', 'funciones',
       'rechazar_candidato', 'contratado_por', 'motivo_de_rechazo',
       'numero_de_documento', 'estado_trabajador', 'pruebas_psicotecnicas',
       'prueba_1', 'prueba_2', 'prueba_3', 'experiencia_-_conocimientos',
       'habilidades', 'contratado', 'pruebas_psicotecnicas_1',
       'desarrollo_familiar_', 'tipo_de_proceso', 'pruebacont_01',
       'pruebacont_02', 'pruebacont_03', 'genero'],
      dtype='object')

In [ ]:
from cgitb import text
import requests
import json
import pandas
import urllib.parse
import urllib3

def token_():
    parameters_= {
        "ClientSecret":"fdaf31a7-d9b6-461b-bf6c-a834d807aa78",
        "ClientId": "e962f0a3-c4ae-4d62-8663-123cd3615ce6"
        }  
    response_=requests.post("https://api.psicoweb.com/login", data = parameters_)
    return json.loads(response_.text)["data"]["token"]

autorizacion = {"Authorization": token_()}
listaCar= []

descargable = requests.get("https://api.psicoweb.com/v1/aplicaciones/link/ExcelConcentrado?prueba=Terman&puesto=NECNOCTerVal", headers = autorizacion)
infoCargos= json.loads(descargable.text).get("data")
linkdes=infoCargos.get("link")
resp = requests.get(linkdes)
archivo="C:\\Users\\CO-166\\Desktop\\pruebas combinaciones\\test.xlsx"

open(archivo, "wb").write(resp.content)
#df = pd.read_excel(archivo, header=6, usecols='B:z')

print(linkdes)

https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOszRC0cKB5VtM4PYrwEqHVdR0fNhIMynhl2Rj7tjm1aolnZyQpXSIQmanCdJ2B1hb%2fNvIpo91TPY1mBn%2bYdXZljtW76ELkvkgPnukTD%2fzGgR2kGsD7kO1XRbBx5TV77P8H2sRIdlogVR%2bcieQoyxkfX4%3d


In [ ]:
from pathlib import Path
archivo="C:\\Users\\CO-166\\Desktop\\pruebas combinaciones\\test.xlsx"
archivoModf= Path.cwd() / archivo
tablaPsicoweb = pd.read_excel(archivoModf, header=5, usecols='B:AZ')
tablaPsicoweb

C:\Users\CO-166\AppData\Local\Temp\ipykernel_1468\1499190619.py:4: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  tablaPsicoweb = pd.read_excel(archivoModf, header=5, usecols='B:AZ')


,Cédula,Nombre,Escolaridad,Edad,Fecha de aplicación,Información (I),Juicio (II),Vocabulario (III),Síntesis (IV),Concentración (V),Análisis (VI),Abstracción (VII),Planeación (VIII),Organización (IX),Atención (X),PN Total,CI,Interpretación,C. Aprendizaje,Compatibilidad
0,1010178224req14599,JOHN ROJAS VILLABON,Bachillerato,27,02/07/2022 09:12:07 AM,10,22,24,14,16,12,9,9,12,12,140,104,NORMAL,NORMAL,-
1,1023954285req14599,CARLOS - MOLANO,Bachillerato,26,09/07/2022 12:26:38 PM,15,22,28,10,16,14,19,13,16,20,173,114,SUPERIOR,SUPERIOR,-
2,1032503454req14599,JUAN - CASTRO,Bachillerato,23,02/07/2022 07:08:34 AM,15,22,22,13,8,16,12,9,12,6,135,103,NORMAL,NORMAL,-
3,1143149688req14599,JESUS AMAYA PERTUZ,Bachillerato,27,01/06/2022 05:24:05 PM,12,16,16,13,14,10,17,3,15,6,122,99,NORMAL,NORMAL,-


In [22]:
from cgitb import text
import requests
import json
import pandas as pd
import urllib.parse
import urllib3
from pathlib import Path

def token_():
    parameters_= {
        "ClientSecret":"fdaf31a7-d9b6-461b-bf6c-a834d807aa78",
        "ClientId": "e962f0a3-c4ae-4d62-8663-123cd3615ce6"
        }  
    response_=requests.post("https://api.psicoweb.com/login", data = parameters_)
    return json.loads(response_.text)["data"]["token"]

autorizacion = {"Authorization": token_()}

linkDescCargo = requests.get("https://api.psicoweb.com/v1/Puestos", headers = autorizacion)
infoCargos= json.loads(linkDescCargo.text).get("data")

listaCar= []

####################################
#"NEC","NIC","NOC",,"PVC"
#listaPrubas=["Cleaver","Valores","FP16","Terman","Zavic"]
listaPrubas=["Cleaver"]

for x in infoCargos:
    var=x["nombre"]
    listaCar.append(urllib.parse.quote(var.encode('utf-8')))
for a in listaPrubas:
    count=0
    for y in listaCar:
        try:
            linkDesc = requests.get("https://api.psicoweb.com/v1/aplicaciones/link/ExcelConcentrado?prueba="+a+"&puesto="+y, headers = autorizacion)

            infoLinkDesc= json.loads(linkDesc.text).get("data")
            linkdes=infoLinkDesc.get("link")
            resp = requests.get(linkdes)

            archivoTemporal="C:\\Users\\CO-166\\Desktop\\pruebas combinaciones\\temporal.xlsx"

            open(archivoTemporal, "wb").write(resp.content)
            count=count+1
            if a == "FP16":
                if count<2:

                    try:    
                        pruebaFP16 = pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        
                    except Exception as e:    
                        print (e)
                else:
                    archivoModf= Path.cwd() / archivoTemporal
                    try:
                        df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        pruebaFP16 = pd.concat([pruebaFP16, df])
                    except Exception as e:    
                        print (e) 
            elif a == "Cleaver":
                if count<2:
                    try:    
                        pruebaCleaver = pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                    except Exception as e:    
                        print (e)
                else:
                    archivoModf= Path.cwd() / archivoTemporal
                    try:
                        df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        pruebaCleaver = pd.concat([pruebaCleaver, df])
                    except Exception as e:    
                        print (e)
            elif a == "PVC":
                if count<2:

                    try:    
                        pruebaPVC = pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                    except:    
                        pass
                else:
                    archivoModf= Path.cwd() / archivoTemporal
                    try:
                        df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        pruebaPVC = pd.concat([pruebaPVC, df])
                    except:
                        pass
                    
            
            elif a == "Valores":
                if count<2:

                    try:    
                        pruebaValores = pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                    except:    
                        pass
                else:
                    archivoModf= Path.cwd() / archivoTemporal
                    try:
                        df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        pruebaValores = pd.concat([pruebaValores, df])
                    except:
                        pass

            elif a == "NEC":
                if count<2:

                    try:    
                        pruebaNEC = pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                    except:    
                        pass
                else:
                    archivoModf= Path.cwd() / archivoTemporal
                    try:
                        df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        pruebaNEC = pd.concat([pruebaNEC, df])
                    except:
                        pass
            elif a == "NIC":
                if count<2:

                    try:    
                        pruebaNIC= pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                    except:    
                        pass
                else:
                    archivoModf= Path.cwd() / archivoTemporal
                    try:
                        df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        pruebaNIC = pd.concat([pruebaNIC, df])
                    except:
                        pass

            elif a == "NOC":
                if count<2:

                    try:    
                        pruebaNOC = pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                    except:    
                        pass
                else:
                    archivoModf= Path.cwd() / archivoTemporal
                    try:
                        df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        pruebaNOC = pd.concat([pruebaNOC, df])
                    except:
                        pass     

            elif a == "Terman":
                if count<2:

                    try:    
                        pruebaTerman = pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                    except:    
                        pass
                else:
                    archivoModf= Path.cwd() / archivoTemporal
                    try:
                        df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        pruebaTerman = pd.concat([pruebaTerman, df])
                    except:
                        pass
                    
            elif a == "Zavic":
                if count<2:

                    try:    
                        pruebaZavic = pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                    except:    
                        pass
                else:
                    archivoModf= Path.cwd() / archivoTemporal
                    try:
                        df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')
                        pruebaZavic = pd.concat([pruebaZavic, df])
                    except:
                        pass                      
            print(linkdes)
        except Exception as e:    
            print (e)
        

C:\Users\CO-166\AppData\Local\Temp\ipykernel_48052\508625080.py:64: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  pruebaCleaver = pd.read_excel(archivoModf, header=5, usecols='B:AZ')


https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs%2bbPbkqXGWwCqCocZQujt8OKUAUUtEayenRvaKo4MWCB6AmHj4JHLARHA4ogPAIeEyfgLCoMxcHk%2fU4niJkOoIrTCg6abT29GgL%2fsSDxOGuE%2fItxe58e3BMepYpEohYbooY2%2fA99BZIA%2bKvQ6rhIwd4%3d
Excel file format cannot be determined, you must specify an engine manually.
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs4InRp70iX12d2JL5Q2b6VcyhnN4p2JYnn4nl83HlNcB7eC%2bSRwHb2TXmb%2bhAQ194fyhfL0M1NhgY6DZuPHdBzNGN1Wu48oo7pYSmng1WhaTn5poA3dvBWmUdsD387pdlZ6IbjYxYWmihn3UdfKrYnE%3d
Excel file format cannot be determined, you must specify an engine manually.
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs8rlutJkXrOTCvyJXNtcvOwCMZTnAnUDGB5%2f70s6OxBK%2fxv1JjpNq%2fey8xzlShIfAs0uSJI8I%2bMIUL98SfR6eEAK8V%2foEVwRr79UfvLSyHaDYgCAPVGWw9EtzZttGFGIwEZPrQCUlvrD8fiPoR2D%2bzQ%3d
Excel file format cannot be determined, you must specify an engine manually.
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs8rlutJkXrOTCvyJXNtcvOzpof1S5IIUUAfKMG7g

C:\Users\CO-166\AppData\Local\Temp\ipykernel_48052\508625080.py:70: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')


https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs7EYKtFsKOXnH1vrG4wqH2rWDogDtTBwcyRZfemILnY0OLu2wzRPwFY7qlymWV4lfp2eKN%2fgkiqGMtM9zxuWHYmrp%2b8Wq71e3MNJvvET6vw%2fVTrgaWJTuDaRK6I%2fJGOy0%2b12MZvxwQqevHM%2fxvzkmOM%3d
Excel file format cannot be determined, you must specify an engine manually.
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs8rlutJkXrOTCvyJXNtcvOypSlh1QC0%2bVSWlKUyeSh16u6q4yr1JLHMVWvK5ic6wKwPgQ38rvOUcksC1vHFlkMjegD1GWPQsr5LTjYFSrb2w%2bPWIkbHvFUW%2f0smtlnopydi3yUU%2f3sicx%2b%2fRzWC623qEVszqYC7wLt1jrGQp30Yj
Excel file format cannot be determined, you must specify an engine manually.
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs8rlutJkXrOTCvyJXNtcvOyBVzKOy4UK7fwbKXq1QyPWk07%2flpMe4bgJIcKt7kWzG34LNwsewH%2bJpimTT8ytlXg3h0LxmlaQHUrTfSaUIxcUcUa3fv54TePJBB4RdYC4QYQyGA8saA8qIB4rSs8uxUg%3d


C:\Users\CO-166\AppData\Local\Temp\ipykernel_48052\508625080.py:70: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')


...

https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs1WOUlEMy2%2b1gWD3YmydtpZzV0reWStLWWGyQa%2fdDDRDjcIDXXqd6QkrOrfhbzdqVZ5Nt7NPyM2KVBj9eqOM2a%2fbzpubAaQbL1Yovp47%2bKvV4Qza0NElxIp%2bq%2fB5FtBV99uZ1xRgFcyYwB3zEYCb4a4%3d
Excel file format cannot be determined, you must specify an engine manually.
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs1WOUlEMy2%2b1gWD3YmydtpbUz0%2bhb%2bB3cgSTlAI5CrgjEGbIJmKHEp667iC5SDD3EKk9NJdfF0VEHBtcHg65DujiUefzVnlmVXeyZGv0GnejZB8fsZ1zpaALJ%2b%2fxoB15FKJS%2bpBWEBRKtazUrmSzi%2fidWSQJ6x9F5PuIKf0P9PtUC31lilFcXatnQcmIXYylJg%3d%3d
Excel file format cannot be determined, you must specify an engine manually.
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs1WOUlEMy2%2b1gWD3YmydtpY%2fz39YevUZP6%2fuJqe8kenquZ3CyO73J8fnjbb9bhSOU6ZAE9kNgYqR%2f4zHFMTBjD%2fQS9%2badWmaeo8fmnTVaCaHmXbOyzt7t8IZiSaduiANJ%2bMAUcct9qsTB2lgZM1tK2U%3d
Excel file format cannot be determined, you must specify an engine manually.
https://plus.psicoweb.com/api?ticket=Jbq%

C:\Users\CO-166\AppData\Local\Temp\ipykernel_48052\508625080.py:70: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')


https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs1WOUlEMy2%2b1gWD3YmydtpbE1mm%2bC2qP%2fsH1sZ0dksIVwccvOaSfAhmKancmHnR4VS2anv2Y%2f7bAjpXz850V9bC58EgYjoL2ALG2JHqr06%2fLkVPjrmWQTi7c%2bpYvTLsRWIKJWZw9FI4ooomR%2bmDhaCpgFeOtfaGk1m1A5NK4anzM
Excel file format cannot be determined, you must specify an engine manually.
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs1WOUlEMy2%2b1gWD3YmydtpZJ0okI%2bBjL2uga1%2b31Dy83jivFN60%2bdiR6993AApOwjKvhsG1X2ZQhiuRP1rcqm%2bvtmeLo7uST8d9E6vaS37ZP%2bsPmrED5R22OL%2bk8rZ967YloV750cty9JqIioP0q4iZvXlvGaEZfX1i6jrC3ExEYzvQnScqb3D4MAD3Var66Yw%3d%3d
Excel file format cannot be determined, you must specify an engine manually.
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs1WOUlEMy2%2b1gWD3YmydtpaECpK4SPUjAi2oiAlPkOGt92%2bWaR8iUkPgOu9eVkUkwoG4BNnXDPHwtzuE%2f6m0D5fLhxr%2bWsliiSyYgrZymIsSJIgF%2bRGZZgkgclUubksnI5jXI8TFMUJbJlwvcJFJvVisgIWZrPdaSuqiI9WS7C5L
Excel file format cannot be determined, you must specify an engine manually.
https:/

C:\Users\CO-166\AppData\Local\Temp\ipykernel_48052\508625080.py:70: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  df=pd.read_excel(archivoModf, header=5, usecols='B:AZ')


https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs2aQ71%2f9zt9OW2%2bxacgOt9bGac2phkTaJNPW8BYIFLcrFNZ9zaDlsiUHIEZEEy3NEEbvx8Va5u74cjOg81QLDiXUvqwoWCBXfbUiLRwyF4PhQ9rf%2fuh%2bFIJYSMYZwvd75nKWQeHV6koE4SfdWOxTa3A%3d
Excel file format cannot be determined, you must specify an engine manually.
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs2aQ71%2f9zt9OW2%2bxacgOt9b7tayzessEaij%2b%2fIOq7Fs7YbcoghyjHTxGgwu4qL%2fPgCZIq4rK1VHmiP0hfDC25kwze0E8Dt8%2bWJIco%2f7vNM18vzAcQQjOlFizAwSL4%2bFPYKuILRM9sJ2ZyQbPwP4qymY%3d
Excel file format cannot be determined, you must specify an engine manually.
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs2aQ71%2f9zt9OW2%2bxacgOt9YZ7Nkl7f7eFVf2PIh3ZbGF1uCLu5%2f3UXcd%2bDarrnZ3lKB%2fsBg8XAdVJNw3Fqcpb%2f33%2f7dBjutN%2fiTx8Roq128r%2be5RW10fejhE3whlwBF2bqXlyWg0skg7UnW5BKWWl78%3d
Excel file format cannot be determined, you must specify an engine manually.
https://plus.psicoweb.com/api?ticket=Jbq%2bCm5Qcp9DAAk78yjOs2aQ71%2f9zt9OW2%2bxacgOt9

In [23]:
pruebaCleaver

,Cédula,Nombre,Escolaridad,Edad,Fecha de aplicación,Combinación,D,I,S,C,...,I.1,S.1,C.1,Compatibilidad.1,D.2,I.2,S.2,C.2,Compatibilidad.2,Válida
0,1012398658,Jhonatan Stiven Gonzalez Ruiz,Secundaria,29,21/02/2022 08:34:19 AM,C/I,48,20,57,90,...,40,40,96,-,69,18,79,74,-,Sí
0,1.010.184.764,enrique carlos badel kerguelen,Maestria,33,24/01/2022 08:05:51 AM,C/I,40,25,75,90,...,58,65,84,-,45,18,89,92,-,Sí
1,1.1015.437.845,Cristian Camilo Hernández Patarroyo,Profesional,29,17/03/2021 04:18:41 PM,C/D,43,79,48,94,...,82,40,93,-,32,75,69,92,-,No
2,1000148925,David Santiago Casas Gómez,Secundaria,22,27/03/2021 01:08:43 PM,C/I,40,10,62,98,...,10,65,98,-,40,18,69,96,-,Sí
3,1000213069,Karla Alejandra Alonso Nuñez,Tecnólogo,21,22/12/2021 08:47:51 AM,C/I,48,43,48,90,...,58,58,93,-,45,45,60,84,-,Sí
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1022341251,DAVID ALBEIRO LASSO GOMEZ,Bachillerato,34,26/04/2021 02:17:01 PM,C/I,52,33,40,90,...,40,40,93,-,60,45,60,84,-,Sí
1,1033736004,Edwin Rubio Diaz,Técnico,31,26/04/2021 02:22:21 PM,C/I,48,15,81,85,...,10,75,73,-,40,30,89,92,-,Sí
2,80798298,JEISSON LEANDRO ROZO JIMENEZ,Tecnólogo,39,26/04/2021 02:45:37 PM,C/S,70,43,33,79,...,58,40,84,-,79,45,40,74,-,Sí
0,1006038112,Santiago Ramirez Gaviria,Técnico,21,02/09/2021 01:42:14 PM,S/I,40,33,85,79,...,40,89,73,-,45,45,79,84,-,Sí


In [72]:
column_names = []
for x in tablaPsicoweb.columns:
     y = normalize(x)
     column_names.append(y)

tablaPsicoweb.columns = column_names
tablaPsicoweb.head(5)

tablaPsicoweb.columns

Index(['cedula', 'nombre', 'escolaridad', 'edad', 'fecha_de_aplicacion',
       'informacion_i', 'juicio_ii', 'vocabulario_iii', 'sintesis_iv',
       'concentracion_v', 'analisis_vi', 'abstraccion_vii', 'planeacion_viii',
       'organizacion_ix', 'atencion_x', 'pn_total', 'ci', 'interpretacion'],
      dtype='object')

for x in tablaPsicoweb.index

In [73]:
listaData=[]
for x in tablaPsicoweb.index:
    if tablaPsicoweb["cedula"][x].__contains__("req") :
        y=tablaPsicoweb["cedula"][x].split("req")
        listaData.append([y[0],y[1]])
    else:
        listaData.append([tablaPsicoweb["cedula"][x], "0"])
df=pd.DataFrame(listaData,columns=["documento","requi"])

joinTable= pd.concat([tablaPsicoweb,df], axis=1)
joinTable


,cedula,nombre,escolaridad,edad,fecha_de_aplicacion,informacion_i,juicio_ii,vocabulario_iii,sintesis_iv,concentracion_v,analisis_vi,abstraccion_vii,planeacion_viii,organizacion_ix,atencion_x,pn_total,ci,interpretacion,documento,requi
0,1010178224req14599,JOHN ROJAS VILLABON,Bachillerato,27,02/07/2022 09:12:07 AM,10,22,24,14,16,12,9,9,12,12,140,104,NORMAL,1010178224,14599
1,1023954285req14599,CARLOS - MOLANO,Bachillerato,26,09/07/2022 12:26:38 PM,15,22,28,10,16,14,19,13,16,20,173,114,SUPERIOR,1023954285,14599
2,1032503454req14599,JUAN - CASTRO,Bachillerato,23,02/07/2022 07:08:34 AM,15,22,22,13,8,16,12,9,12,6,135,103,NORMAL,1032503454,14599
3,1143149688req14599,JESUS AMAYA PERTUZ,Bachillerato,27,01/06/2022 05:24:05 PM,12,16,16,13,14,10,17,3,15,6,122,99,NORMAL,1143149688,14599


In [75]:
tabla2.join(joinTable.set_index('documento'), on='documento')

,nombres_y_apellidos,fecha_de_nacimiento,genero_de_nacimiento,estado_civil,nivel_educativo,empresa,sede,cargo,ciudad_de_residencia,tipo_de_documento,...,concentracion_v,analisis_vi,abstraccion_vii,planeacion_viii,organizacion_ix,atencion_x,pn_total,ci,interpretacion,requi
0,JAIME ALBERTO RAMIREZ CUERVO,13/07/1995,MASCULINO,NaN,0,SECURITY SHOPS LIMITADA,SECURITY SHOPS - SEDE PRINCIPAL BOGOTA,TÉCNICO DE SOSTENIMIENTO,"BOGOTA, D.C.",CEDULA DE CIUDADANIA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JAIDER NAYID LOPEZ BLANCO,02/06/2003,MASCULINO,SOLTERO(A),2,OXIGENO CERO GRADOS S.A.S,OXIGENO CERO GRADOS - SEDE PRINCIPAL BOGOTA,BODEGUERO,"BOGOTA, D.C.",CEDULA DE CIUDADANIA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,YORYIS BARROS GUTIÉRREZ,04/04/1997,FEMENINO,SOLTERO(A),2,DAMIS SAS,BARRANQUILLA,INSTALADOR,BARRANQUILLA,CEDULA DE CIUDADANIA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OSWALDO ANTINIO JIMENEZ RIBON,14/12/1996,MASCULINO,SOLTERO(A),3,DAMIS SAS,BARRANQUILLA,INSTALADOR,BARRANQUILLA,CEDULA DE CIUDADANIA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,YESSIKA JULIANA PACHECO BETANCUR,27/10/1995,FEMENINO,SOLTERO(A),3,AUDIFARMA S.A.,ALCALA,AUXILIAR CAF,ALCALA,CEDULA DE CIUDADANIA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79442,RONALD DAVIAN DELGADO VEGA,21/11/1988,MASCULINO,UNION LIBRE,0,TREFILADOS DE COLOMBIA S.A.S.,BODEGA MOSQUERA,OPERARIO,MOSQUERA,CEDULA DE CIUDADANIA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79443,ALFONSO RAFAEL CANDAMA BROCHERO,14/09/1983,MASCULINO,SOLTERO(A),0,TREFILADOS DE COLOMBIA S.A.S.,BODEGA MOSQUERA,OPERARIO,MOSQUERA,CEDULA DE CIUDADANIA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79444,Jhonathan Danilo Buyucue Mosquera,01/06/1993,MASCULINO,UNION LIBRE,0,TREFILADOS DE COLOMBIA S.A.S.,BODEGA MOSQUERA,OPERARIO,MOSQUERA,CEDULA DE CIUDADANIA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79445,HAROLD ANDRES ALZATE GONZALEZ,30/01/1984,MASCULINO,UNION LIBRE,2,TREFILADOS DE COLOMBIA S.A.S.,BODEGA MOSQUERA,OPERARIO,MOSQUERA,CEDULA DE CIUDADANIA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
